In [356]:
import pandas
import os
import re

In [357]:
SAMPLE = 'sample_0'
SOLVED_FILE_PATH = 'data/output/solved/'
EQUATIONS_FILE_PATH = ''

In [358]:
ground = pandas.read_json('data/input/draw.json')
ground = ground[['lSolutions', 'iIndex']]
ground = ground.set_index('iIndex')
display(len(ground))
ground.head(1)

999

,lSolutions
iIndex,
397760,[2.14285714286]


In [359]:
for file_path in os.listdir(SOLVED_FILE_PATH):
    solved = pandas.read_json(f"{SOLVED_FILE_PATH}{file_path}", lines=True)
    solved = solved[['question_id', 'question', 'choices']]
    solved['choices'] = solved['choices'].apply(lambda row : row[0]['message']['content'])
    file_name = file_path.split('.')[0]
    solved = solved.rename(columns={'question': f'{file_name}__responses', 'choices': f'{file_name}__equations'})
    solved = solved.set_index('question_id')
    ground = ground.join(solved, how='left')

# print(len(ground))
ground = ground[~ground.index.duplicated(keep='first')]
ground

,lSolutions,sample_0__responses,sample_0__equations,sample_1__responses,sample_1__equations,sample_2__responses,sample_2__equations,sample_3__responses,sample_3__equations,sample_4__responses,...,sample_5__responses,sample_5__equations,sample_6__responses,sample_6__equations,sample_7__responses,sample_7__equations,sample_8__responses,sample_8__equations,sample_9__responses,sample_9__equations
iIndex,,,,,,,,,,,,,,,,,,,,,
337,[2.4],Extract the equations from this response.\nYou...,1/4x + 1/6x = 1\n3x + 2x = 12\n5x = 12\nx = 2.4,Extract the equations from this response.\nYou...,1/4 + 1/6 = 1/x\n5/12 = 1/x\nx = 12/5,Extract the equations from this response.\nYou...,4 = 1/R_sara\n6 = 1/R_john\nR_combined = R_sar...,Extract the equations from this response.\nYou...,1 = (1/4)t + (1/6)t \nt = 12/5,Extract the equations from this response.\nYou...,...,Extract the equations from this response.\nYou...,S = Work / 4\nJ = Work / 6\n(S + J) = Work / T...,Extract the equations from this response.\nYou...,1/4 = Work done / Time taken \n1/6 = Work done...,Extract the equations from this response.\nYou...,1/4 + 1/6 = 1/x\n5x = 12\nx = 12/5,Extract the equations from this response.\nYou...,time = work / rate\nx = 1 / (5/12)\nx = 12/5,Extract the equations from this response.\nYou...,1 house / 4 hours = Sara's rate\n1 house / 6 h...
1042,"[9.0, 12.0]",Extract the equations from this response.\nYou...,3x = 4y\nx + y = 21\ny = (3/4)x\nx + (3/4)x = ...,Extract the equations from this response.\nYou...,3x = 4y \nx + y = 21,Extract the equations from this response.\nYou...,3y = 4x \nx + y = 21,Extract the equations from this response.\nYou...,3x = 4y \nx + y = 21 \ny = 21 - x \n3x = 4(21 ...,Extract the equations from this response.\nYou...,...,Extract the equations from this response.\nYou...,3x = 4y\nx + y = 21,Extract the equations from this response.\nYou...,3x = 4y\nx + y = 21,Extract the equations from this response.\nYou...,3x = 4y\nx + y = 21,Extract the equations from this response.\nYou...,3x = 4y \nx + y = 21,Extract the equations from this response.\nYou...,3x = 4y\nx + y = 21\ny = (3/4)x\n(7/4)x = 21\n...
1299,"[12.5, 87.5]",Extract the equations from this response.\nYou...,y = 7x\nx + y = 100\n8x = 100\nx = 12.50\ny = ...,Extract the equations from this response.\nYou...,h + s = 100\nh = 7s\n7s + s = 100\ns = 12.50\n...,Extract the equations from this response.\nYou...,x + 7x = 100\n8x = 100\nx = 12.5,Extract the equations from this response.\nYou...,x + 7x = 100.00\n8x = 100.00\nx = 12.50\n7x = ...,Extract the equations from this response.\nYou...,...,Extract the equations from this response.\nYou...,x + 7x = 100\n8x = 100\nx = 12.50,Extract the equations from this response.\nYou...,x + 7x = 100\n8x = 100\nx = 12.50,Extract the equations from this response.\nYou...,H + S = 100 \nH = 7S,Extract the equations from this response.\nYou...,x + 7x = 100\n8x = 100\nx = 12.50\n7x = 87.50,Extract the equations from this response.\nYou...,x + 7x = $100.00\n8x = $100.00\nx = $12.50\nVa...
1422,[25.0],Extract the equations from this response.\nYou...,0.60(100) + 1.10x = 0.70(100 + x)\n0.4x = 10\n...,Extract the equations from this response.\nYou...,(60 + 1.1x)/(100 + x) = 0.70 \n60 + 1.1x = 70 ...,Extract the equations from this response.\nYou...,0.60(100) + 1.10x = 0.70(100 + x)\n0.40x = 10\...,Extract the equations from this response.\nYou...,100(0.60) + x(1.10) = (100 + x)(0.70)\n60 + 1....,Extract the equations from this response.\nYou...,...,Extract the equations from this response.\nYou...,100 * 0.60 + x * 1.10 = (100 + x) * 0.70\n0.4x...,Extract the equations from this response.\nYou...,Total cost = 100(0.60) + x(1.10)\nTotal weight...,Extract the equations from this response.\nYou...,1. 60 + 1.1x = 0.7(100 + x)\n2. 0.4x = 10\n3. ...,Extract the equations from this response.\nYou...,0.60(100) + 1.10x = 0.70(100 + x)\n0.40x = 10\...,Extract the equations from this response.\nYou...,100(0.60) + x(1.10) = (100 + x)(0.70)\n0.4x = ...
1431,"[4.0,

In [ ]:
def label_category(row, sample_name):
    category = row.name[0]
    if category > 8: 
        row['E. Final answer and equations same'] = 1
        category -= 8
    else: row['E. Final answer and equations same'] = 0

    if category == 1: 
        row['Equations'] = '''UNKNOWN'''
        row['Solving'] = '''FAIL'''
        row['Theory'] = '''Definitely cannot solve, may have got wrong rep.'''
        row['Understandable'] = '''YES'''
    elif category == 2: 
        row['Equations'] = '''SUCCEED'''
        row['Solving'] = '''FAIL'''
        row['Theory'] = '''Gets equations right, but solves wrong'''
        row['Understandable'] = '''YES'''
    elif category == 3: 
        row['Equations'] = '''FAIL'''
        row['Solving'] = '''SUCCEED'''
        row['Theory'] = '''Gets equations wrong, but it can solve otherwise.'''
        row['Understandable'] = '''YES'''
    elif category == 4: 
        row['Equations'] = '''SUCCEED'''
        row['Solving'] = '''SUCCEED'''
        row['Theory'] = '''Non-determinism issues as we do not know why it can't handle by itself'''
        row['Understandable'] = '''NO'''
    elif category == 5: 
        row['Equations'] = '''UNKNOWN'''
        row['Solving'] = '''FAIL'''
        row['Theory'] = '''Non-determinism as we do not know how it could gotten the problem right'''
        row['Understandable'] = '''NO'''
    elif category == 6: 
        row['Equations'] = '''SUCCEED'''
        row['Solving'] = '''FAIL'''
        row['Theory'] = '''Can get the right equations, perhaps there is something about the equations from ground truth that mess it up'''
        row['Understandable'] = '''MAYBE'''
    elif category == 7: 
        row['Equations'] = '''SUCCEED'''
        row['Solving'] = '''SUCCEED'''
        row['Theory'] = '''ChatGPT is lying on the equations it reports'''
        row['Understandable'] = '''MAYBE'''
    elif category == 8: 
        row['Equations'] = '''SUCCEED'''
        row['Solving'] = '''SUCCEED'''
        row['Theory'] = '''Gets everything right'''
        row['Understandable'] = '''YES'''
    return row

values = pandas.DataFrame(joined[[
    'category', 
    'A. ChatGPT gets it correct from text to final solution', 
    'B. Sympy gets it correct from equations to final solution',
    'C. ChatGPT gets it correct from equations to final solution',
    'D. Gets right answer when ChatGPT transforms text to equations and SymPy solves the equations from chat GPT']].value_counts().sort_index())
values = values.apply(lambda row : label_category(row), axis=1)
values = values[[
    'E. Final answer and equations same',
    'count',
    'Equations',
    'Solving',
    'Understandable',
    'Theory'
]]
values

,,,,,E. Final answer and equations same,count,Equations,Solving,Understandable,Theory
category,A. ChatGPT gets it correct from text to final solution,B. Sympy gets it correct from equations to final solution,C. ChatGPT gets it correct from equations to final solution,D. Gets right answer when ChatGPT transforms text to equations and SymPy solves the equations from chat GPT,,,,,,
1,0,1,0,0,0,129,UNKNOWN,FAIL,YES,"Definitely cannot solve, may have got wrong rep."
2,0,1,0,1,0,53,SUCCEED,FAIL,YES,"Gets equations right, but solves wrong"
3,0,1,1,0,0,63,FAIL,SUCCEED,YES,"Gets equations wrong, but it can solve otherwise."
4,0,1,1,1,0,63,SUCCEED,SUCCEED,NO,Non-determinism issues as we do not know why i...
5,1,1,0,0,0,9,UNKNOWN,FAIL,NO,Non-determinism as we do not know how it could...
6,1,1,0,1,0,3,SUCCEED,FAIL,MAYBE,"Can get the right equations, perhaps there is ..."
7,1,1,1,0,0,31,SUCCEED,SUCCEED,MAYBE,ChatGPT is lying on the equations it reports
8,1,1,1,1,0,8,SUCCEED,SUCCEED,YES,Gets everything right
9,0,1,0,0,1,46,UNKNOWN,FAIL,YES,"Definitely cannot solve, may have got wrong rep."


In [ ]:
import xlsxwriter

writer = pandas.ExcelWriter('data/output/xlsx/categories.xlsx', engine='xlsxwriter')
values.to_excel(writer, sheet_name='List')

workbook = writer.book

wrap_format = workbook.add_format()
wrap_format.set_text_wrap()

list_sheet = writer.sheets['List']
list_sheet.set_column('A:H', 15, wrap_format)
pie_chart = workbook.add_chart({'type': 'pie'})

pie_chart.set_style(10)
pie_chart.set_size({'width': 500, 'height': 300})
pie_chart.set_title({'name': f'ChatGPT stats'})
pie_chart.add_series({
    'name': f'ChatGPT stats',
    'categories': f"=Chart!$A$2:$A$10",
    'values': f"=Chart!$G$2:$G$16",
    'points': [
        {'fill': {'color': '#e7366b'}},
        {'fill': {'color': '#29cc7a'}},
        {'fill': {'color': '#ffc619'}},
    ],
    'data_labels': {'value': True, 'category': True, 'position': 'center', 'percentage': True, 'name_font': {'size': 14}}
})
list_sheet.insert_chart('A20', pie_chart, {'x_offset': 0, 'y_offset': 0})
workbook.close()


C:\Users\nocet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xlsxwriter\workbook.py:1660: UserWarning: Unknown worksheet reference 'Chart' in range 'Chart!$A$2:$A$10' passed to add_series()
  warn("Unknown worksheet reference '%s' in range "
C:\Users\nocet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xlsxwriter\workbook.py:1660: UserWarning: Unknown worksheet reference 'Chart' in range 'Chart!$G$2:$G$16' passed to add_series()
  warn("Unknown worksheet reference '%s' in range "
